In [137]:
%load_ext autoreload
%autoreload 2
%pylab inline

import sys
import glob
import pandas as pd
import os
import seaborn as sns
# from tqdm import tqdm_notebook as tqdm
from tqdm import tqdm
import pickle
from statsmodels.distributions.empirical_distribution import ECDF
from collections import defaultdict
import logging
from open_spiel.python.examples.ubc_mccfr_cpp_example import action_to_bids
from open_spiel.python.examples.ubc_nfsp_example import policy_from_checkpoint
from open_spiel.python.examples.ubc_utils import *

from open_spiel.python.pytorch.ubc_nfsp import NFSP
import bokeh
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import HoverTool, ColumnDataSource, ColorBar, LogColorMapper, LinearColorMapper
from bokeh.transform import linear_cmap, log_cmap

output_notebook()
# logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.DEBUG, datefmt='%I:%M:%S')
# logger = logging.getLogger()
# logger.setLevel(logging.INFO)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Populating the interactive namespace from numpy and matplotlib


/apps/open_spiel/venv/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['show', 'figure', 'logging']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


Loading BokehJS ...

In [138]:
EXPERIMENT_DIR = '/shared/outputs/big_game_2/lstm_deep'

In [139]:
CHECKPOINT_NAME = 'checkpoint_latest'
checkpoint = policy_from_checkpoint(EXPERIMENT_DIR, CHECKPOINT_NAME)

In [140]:
def check_on_q_values(checkpoint, agent, state=None):
    q_network = checkpoint.agents[agent]._rl_agent._q_network
    if state is None:
        state = checkpoint.game.new_initial_state().child(0).child(0)
    legal_actions = state.legal_actions()
    it = state.information_state_tensor()
    info_state = q_network.prep_batch([q_network.reshape_infostate(it)])
    q_values = q_network(info_state).detach()[0]
    legal_q_values = q_values[legal_actions]
    action_dict = get_actions(checkpoint.game)
    return {s: q for s,q in zip(action_dict.values(), legal_q_values)}

In [141]:
check_on_q_values(checkpoint, 0)

{'Bid for 0,0,0 licenses @ $0 with activity 0': tensor(1.1621),
 'Bid for 0,0,1 licenses @ $25 with activity 25': tensor(9.7418),
 'Bid for 0,1,0 licenses @ $50 with activity 50': tensor(6.5731),
 'Bid for 0,1,1 licenses @ $75 with activity 75': tensor(9.3640),
 'Bid for 1,0,0 licenses @ $75 with activity 75': tensor(45.0049),
 'Bid for 1,0,1 licenses @ $100 with activity 100': tensor(14.9747),
 'Bid for 1,1,0 licenses @ $125 with activity 125': tensor(9.5806),
 'Bid for 1,1,1 licenses @ $150 with activity 150': tensor(7.7787),
 'Bid for 2,0,0 licenses @ $150 with activity 150': tensor(10.6129),
 'Bid for 2,0,1 licenses @ $175 with activity 175': tensor(6.6275),
 'Bid for 2,1,0 licenses @ $200 with activity 200': tensor(5.8766),
 'Bid for 2,1,1 licenses @ $225 with activity 225': tensor(5.0969)}

In [142]:
check_on_q_values(checkpoint, 1, checkpoint.game.new_initial_state().child(0).child(0).child(4))

{'Bid for 0,0,0 licenses @ $0 with activity 0': tensor(1.4211),
 'Bid for 0,0,1 licenses @ $25 with activity 25': tensor(11.3263),
 'Bid for 0,1,0 licenses @ $50 with activity 50': tensor(8.9059),
 'Bid for 0,1,1 licenses @ $75 with activity 75': tensor(11.0143),
 'Bid for 1,0,0 licenses @ $75 with activity 75': tensor(10.2344),
 'Bid for 1,0,1 licenses @ $100 with activity 100': tensor(10.9816),
 'Bid for 1,1,0 licenses @ $125 with activity 125': tensor(104.9926),
 'Bid for 1,1,1 licenses @ $150 with activity 150': tensor(11.7331),
 'Bid for 2,0,0 licenses @ $150 with activity 150': tensor(11.1616),
 'Bid for 2,0,1 licenses @ $175 with activity 175': tensor(7.7227),
 'Bid for 2,1,0 licenses @ $200 with activity 200': tensor(7.2141),
 'Bid for 2,1,1 licenses @ $225 with activity 225': tensor(7.7755)}

In [143]:
time_step = checkpoint.env.reset()

In [144]:
time_step.observations['current_player']

0

In [145]:
time_step = checkpoint.env.step([5])
# time_step = checkpoint.env.step(action_list)

In [146]:
time_step.observations['current_player']

1

In [147]:
time_step = checkpoint.env.step([6])

In [148]:
dir(time_step)

['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getnewargs__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmul__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '_asdict',
 '_fields',
 '_make',
 '_replace',
 '_source',
 'count',
 'current_player',
 'discounts',
 'first',
 'index',
 'is_simultaneous_move',
 'last',
 'mid',
 'observations',
 'rewards',
 'step_type']

In [149]:
time_step.rewards

[50.0, 55.0]

In [150]:
checkpoint.agents[0]._rl_agent._optimizer.param_groups[0]['lr'] = 1e-3

In [151]:
checkpoint.agents[0]._rl_agent._batch_size = 1000

In [152]:
checkpoint.agents[0]._rl_agent._epsilon_start = 0.5

In [153]:
actions = []
for episode in tqdm(range(1000000)):
    if episode % 1000 == 0:
        qs = check_on_q_values(checkpoint, 0)
        print("Bid for 1,0,0 licenses @ $75 with activity 75 -", qs["Bid for 1,0,0 licenses @ $75 with activity 75"])
        print("Bid for 1,0,1 licenses @ $100 with activity 100 -", qs["Bid for 1,0,1 licenses @ $100 with activity 100"])
        print(pd.Series(actions).value_counts())
        
    time_step = checkpoint.env.reset()
    first_action = True

    while not time_step.last():
        player_id = time_step.observations["current_player"]
        agent = checkpoint.agents[player_id]
        if player_id == 0:
            agent._best_response_mode = True
        # Player 1 is fixed
        agent_output = agent.step(time_step, is_evaluation = player_id==1) 
#         if player_id == 0 and agent_output.action != 4:
#             print(agent_output.action)
        
        if player_id == 0 and first_action:
            actions.append(agent_output.action)
            first_action = False
        action_list = [agent_output.action]
        time_step = checkpoint.env.step(action_list)

    # Episode is over, step all agents with final info state.
    for player_id, agent in enumerate(checkpoint.agents):
        agent.step(time_step)
    

  0%|          | 0/1000000 [00:00<?, ?it/s]/apps/open_spiel/venv/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  import sys
  0%|          | 12/1000000 [00:00<2:45:10, 100.90it/s]

Bid for 1,0,0 licenses @ $75 with activity 75 - tensor(45.0049)
Bid for 1,0,1 licenses @ $100 with activity 100 - tensor(14.9747)
Series([], dtype: int64)


  0%|          | 1021/1000000 [00:06<1:48:06, 154.02it/s]

Bid for 1,0,0 licenses @ $75 with activity 75 - tensor(45.0049)
Bid for 1,0,1 licenses @ $100 with activity 100 - tensor(14.9747)
4     575
11     52
1      48
9      39
5      39
3      39
2      38
8      35
10     34
7      34
6      34
0      33
dtype: int64


  0%|          | 2017/1000000 [00:12<1:42:30, 162.26it/s]

...

Bid for 1,0,0 licenses @ $75 with activity 75 - tensor(37.2150)
Bid for 1,0,1 licenses @ $100 with activity 100 - tensor(49.9940)
5     43179
4      9856
6      8925
11     6768
10     5317
3      4118
9      4029
0      4012
8      4009
1      3982
2      3914
7      3891
dtype: int64


 10%|█         | 103004/1000000 [3:05:45<19:17:38, 12.91it/s]

Bid for 1,0,0 licenses @ $75 with activity 75 - tensor(37.3752)
Bid for 1,0,1 licenses @ $100 with activity 100 - tensor(50.0288)
5     43777
4      9892
6      8974
11     6810
10     5362
3      4161
9      4057
0      4043
8      4036
1      4012
2      3946
7      3930
dtype: int64


 10%|█         | 103448/1000000 [3:06:17<26:54:35,  9.25it/s]


KeyboardInterrupt: 

In [125]:
[(i, t) for (i, t) in enumerate(checkpoint.agents[0]._rl_agent.replay_buffer._data) if t.action == 5]

[(2826,
  Transition(info_state=tensor([[  1.,   0.,   1.,   0.,   1.,   0., 300., 100.,  70.,  50.,   0.,   0.,
             0.,   0.,   0.,   0.,   0.,   0.,   0.,  75.,  50.,  25.]]), action=5, reward=50.0, next_info_state=tensor([[  0.,   0.,   1.,   0.,   1.,   0., 300., 100.,  70.,  50.,   1.,   0.,
             1.,   1.,   0.,   1.,   2.,   1.,   1.,  75.,  50.,  25.]]), is_final_step=1.0, legal_actions_mask=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), iteration=2802)),
 (3182,
  Transition(info_state=tensor([[  1.,   0.,   1.,   0.,   1.,   0., 300., 100.,  70.,  50.,   0.,   0.,
             0.,   0.,   0.,   0.,   0.,   0.,   0.,  75.,  50.,  25.]]), action=5, reward=50.0, next_info_state=tensor([[  0.,   0.,   1.,   0.,   1.,   0., 300., 100.,  70.,  50.,   1.,   0.,
             1.,   1.,   0.,   1.,   2.,   1.,   1.,  75.,  50.,  25.]]), is_final_step=1.0, legal_actions_mask=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), iteration=3157))]

In [126]:
[(i, t) for (i, t) in enumerate(checkpoint.agents[0]._rl_agent.replay_buffer._data) if t.action == 4][:1]

[(0,
  Transition(info_state=tensor([[  1.,   0.,   1.,   0.,   1.,   0., 300., 120.,  80.,  30.,   0.,   0.,
             0.,   0.,   0.,   0.,   0.,   0.,   0.,  75.,  50.,  25.]]), action=4, reward=45.0, next_info_state=tensor([[  0.,   0.,   1.,   0.,   1.,   0., 300., 120.,  80.,  30.,   1.,   0.,
             0.,   1.,   0.,   0.,   2.,   1.,   0.,  75.,  50.,  25.]]), is_final_step=1.0, legal_actions_mask=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), iteration=0))]

In [128]:
good_transition_1 = checkpoint.agents[0]._rl_agent.replay_buffer._data[2826]
good_transition_2 = checkpoint.agents[0]._rl_agent.replay_buffer._data[0]

In [129]:
checkpoint.agents[0]._rl_agent.replay_buffer._data = [good_transition_1 for _ in range(2500)] + [good_transition_2 for _ in range(2500)]

In [130]:
for iteration in range(100):
    loss = checkpoint.agents[0]._rl_agent.learn()
    print(loss.detach().item())
#     print(loss)
    
    if iteration % 10 == 0:
        qs = check_on_q_values(checkpoint, 0)
        print("Bid for 1,0,0 licenses @ $75 with activity 75 -", qs["Bid for 1,0,0 licenses @ $75 with activity 75"])
        print("Bid for 1,0,1 licenses @ $100 with activity 100 -", qs["Bid for 1,0,1 licenses @ $100 with activity 100"])

807.5289306640625
Bid for 1,0,0 licenses @ $75 with activity 75 - tensor(45.8867)
Bid for 1,0,1 licenses @ $100 with activity 100 - tensor(16.0223)
586.7173461914062
543.442138671875
514.162841796875
507.9754638671875
436.6761169433594
417.8582763671875
371.98883056640625
340.9349670410156
324.2243957519531
277.7006530761719
Bid for 1,0,0 licenses @ $75 with activity 75 - tensor(46.6113)
Bid for 1,0,1 licenses @ $100 with activity 100 - tensor(26.9745)
279.0695495605469
239.5868377685547
225.343505859375
195.4790496826172
179.09288024902344
183.2227325439453
163.67933654785156
151.09359741210938
134.9404754638672
120.89849853515625
Bid for 1,0,0 licenses @ $75 with activity 75 - tensor(47.2011)
Bid for 1,0,1 licenses @ $100 with activity 100 - tensor(35.2648)
109.08171844482422
100.81904602050781
97.08745574951172
84.25435638427734
75.76222229003906
69.11871337890625
62.33546829223633
60.39402770996094
49.9107780456543
50.1098518371582
Bid for 1,0,0 licenses @ $75 with activity 75 - te

In [ ]:
[d for d in checkpoint.agents[0]._rl_agent._replay_buffer._data if d.reward in [50., 30.]]

In [ ]:
pd.DataFrame(q).drop_duplicates()

In [ ]:
df = pd.DataFrame.from_records(records)

In [ ]:
df = df.drop_duplicates(subset=['infostate_string'])
print(f'After dropping, {len(df)} unique infostates remaining')